In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [4]:
df=pd.read_csv('diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
x=df.drop(columns=['Outcome']).values
y=df['Outcome'].values

In [8]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2)


In [9]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [10]:
# scaling the data 

In [11]:
from sklearn.preprocessing import  StandardScaler
scale=StandardScaler()
xtrain=scale.fit_transform(xtrain)
xtest=scale.fit_transform(xtest)

# Model without keras tuner

In [12]:
temp_model=Sequential()
temp_model.add(Dense(256,activation='relu',input_dim=8))
temp_model.add(Dense(128,activation='relu'))
temp_model.add(Dense(1,activation='sigmoid'))
temp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2304      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 35329 (138.00 KB)
Trainable params: 35329 (138.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
temp_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
temp_model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50)

Epoch 1/50


20/20 [==============================] - 1s 8ms/step - loss: 0.5616 - accuracy: 0.7362 - val_loss: 0.4895 - val_accuracy: 0.7662
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4725 - accuracy: 0.7785 - val_loss: 0.4674 - val_accuracy: 0.7792
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4503 - accuracy: 0.7834 - val_loss: 0.4529 - val_accuracy: 0.7792
Epoch 4/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.7866 - val_loss: 0.4491 - val_accuracy: 0.7662
Epoch 5/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4343 - accuracy: 0.7834 - val_loss: 0.4608 - val_accuracy: 0.7792
Epoch 6/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4322 - accuracy: 0.7915 - val_loss: 0.4522 - val_accuracy: 0.7597
Epoch 7/50
20/20 [==============================] - 0s 2ms/step - loss: 0.4317 - accuracy: 0.8046 - val_loss: 0.4633 - val_accuracy: 0.7792
Epoch 8/50
20/20 [=============

So problems in this model is we dont able to know:

- how many layers we have to add

- how many node a particular layer have

- which loss function is best for each layer

- which activation function is suitable for every layer

- how many epoch the model have to train 

Here the keras tuner is came into the picture

# Model with keras tuner

In [15]:
import kerastuner as kt

C:\Users\Asus\AppData\Local\Temp\ipykernel_11852\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


There are 6 steps 

- first is you have to make a function named "build_model(hp)" // where hp is the object(hyperparameter)

- second is you have to make "tuner object" of a class RandomSearch()

- you have to call the "search()" function using tuner object

- extract best hyperparameter 

- Get Best model

- make the model using the best hyperparameter we got

## how to tune the appropriate optimizer

- step1: build_model()

In [16]:
def build_model(hp):
    """build a model"""
    model=Sequential()
    model.add(Dense(256,activation='relu',input_dim=8))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']) # a function which takes two thing 1st is string and 2nd is list
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

- step2: make a tuner object of class RandomSearch

In [17]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='bestOptimizer') #project_name making a folder containing the info

Reloading Tuner from .\bestOptimizer\tuner0.json


- step3: call search() using tuner object

In [18]:
tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

- step4: get best hyperparameter

In [19]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

- step5: Get best model 

In [20]:
model=tuner.get_best_models(num_models=1)[0]

- step6: train the model

In [21]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),initial_epoch=6,epochs=100) # initial_epoch is 6 because we already run 5 epochs

Epoch 7/100


20/20 [==============================] - 1s 16ms/step - loss: 0.4716 - accuracy: 0.7850 - val_loss: 0.4469 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4648 - accuracy: 0.7736 - val_loss: 0.4534 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4469 - accuracy: 0.7818 - val_loss: 0.4371 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4438 - accuracy: 0.7915 - val_loss: 0.4402 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4351 - accuracy: 0.7899 - val_loss: 0.4398 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4298 - accuracy: 0.7980 - val_loss: 0.4382 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4266 - accuracy: 0.8078 - val_loss: 0.4457 - val_accuracy: 0.7922
Epoch 14/100
20/20 [

# Tune no of Nodes in each layer

- step1: build_model

In [22]:
def build_model(hp):
    """ build the model (its not working when the multiple layers is used (we can check further))"""
    model = Sequential()
    units = hp.Int('units', min_value=32, max_value=256)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    # model.add(Dense(units=units, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


- step2: keras tuner object of randomsearch class 

In [23]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='best no of nodes')

Reloading Tuner from .\best no of nodes\tuner0.json


- step3: call search using tuner object 

In [24]:
tuner.search(xtrain,ytrain,validation_data=(xtest,ytest),epochs=5)

- step 4: get best parameter

In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 192}

- step5: extracting the model

In [26]:
model=tuner.get_best_models(num_models=1)[0]

- step 6: extract and train the model

In [27]:
model.fit(xtrain,ytrain,initial_epoch=6,epochs=100 ,validation_data=(xtest,ytest))

Epoch 7/100


20/20 [==============================] - 1s 18ms/step - loss: 0.4874 - accuracy: 0.7752 - val_loss: 0.4583 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4727 - accuracy: 0.7736 - val_loss: 0.4535 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4637 - accuracy: 0.7834 - val_loss: 0.4484 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4580 - accuracy: 0.7899 - val_loss: 0.4495 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4536 - accuracy: 0.7883 - val_loss: 0.4496 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4498 - accuracy: 0.7866 - val_loss: 0.4524 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4483 - accuracy: 0.7899 - val_loss: 0.4518 - val_accuracy: 0.7662
Epoch 14/100
20/20 [

# Tune numbers of layers in a model

- step1: build_model()

In [28]:
def build_model(hp):
    """ So here we have to run a loop to check how many layers model have performing the best"""
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(256,activation='relu',input_dim=8))
            counter+=1
        else:
            model.add(Dense(128,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
    return model

- step2: make a object of keras tuner 

In [29]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='no of layers')

Reloading Tuner from .\no of layers\tuner0.json


- step3: call the search function using tuner object

In [30]:
tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

- step4: find best hyperparameter 

In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

- step5: extracting model

In [32]:
model=tuner.get_best_models(num_models=1)[0]

- step6: training the model

In [33]:
model.fit(xtrain,ytrain,epochs=100,initial_epoch=5,validation_data=(xtest,ytest))

Epoch 6/100
20/20 [==============================] - 2s 19ms/step - loss: 0.5927 - accuracy: 0.6954 - val_loss: 0.4762 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4956 - accuracy: 0.7752 - val_loss: 0.5468 - val_accuracy: 0.7338
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4712 - accuracy: 0.7785 - val_loss: 0.4549 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4595 - accuracy: 0.7720 - val_loss: 0.4541 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4534 - accuracy: 0.7785 - val_loss: 0.4886 - val_accuracy: 0.7532
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4413 - accuracy: 0.7997 - val_loss: 0.5694 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4378 - accuracy: 0.7948 - val_loss: 0.4565 - val_accuracy: 0.7792
Epoch 13/

# Tune every hyperparameter of a model

- step1: make build_model()

In [51]:
def build_model(hp):
    """ In this run a loop stores the every hyperparmater using hp object """
    model=Sequential()
    counter=0
    for i in range(hp.Int('no of layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(
                hp.Int('no of nodes'+str(i),min_value=32,max_value=256),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','selu','elu']),
                input_dim=8
            ))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
            counter+=1
        else:
            model.add(Dense
                (hp.Int('no of nodes'+str(i),min_value=32,max_value=256),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','selu','elu']),
            ))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('best optimizer',values=['adam','rmsprop','adadelta','sgd','momentum']),
        loss='binary_crossentropy', # you can also give here the different loss function using hp.Choice() 
        metrics=['accuracy']
    )
    return model

- step2: make tuner object of RandomSearch class

In [52]:
tuner=kt.RandomSearch(build_model,objective='accuracy',max_trials=5,project_name='Tunning all best hyperparamters')

Reloading Tuner from .\Tunning all best hyperparamters\tuner0.json


- step3: call search() using tuner object

In [53]:
tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

- step4: getting best hyperparameters

In [54]:
tuner.get_best_hyperparameters()[0].values

{'no of layers': 6,
 'no of nodes0': 124,
 'activation0': 'sigmoid',
 'best optimizer': 'sgd',
 'no of nodes1': 227,
 'activation1': 'relu',
 'no of nodes2': 59,
 'activation2': 'elu',
 'no of nodes3': 99,
 'activation3': 'selu',
 'no of nodes4': 162,
 'activation4': 'tanh',
 'no of nodes5': 32,
 'activation5': 'relu'}

- step5 : extracting best model

In [55]:
model=tuner.get_best_models(num_models=1)[0]

- step6: model training

In [56]:
model.fit(xtrain,ytrain,epochs=200,initial_epoch=5,validation_data=(xtest,ytest))

Epoch 6/200
20/20 [==============================] - 2s 22ms/step - loss: 0.6283 - accuracy: 0.6515 - val_loss: 0.5536 - val_accuracy: 0.6948
Epoch 7/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6330 - accuracy: 0.6645 - val_loss: 0.5455 - val_accuracy: 0.7143
Epoch 8/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6143 - accuracy: 0.6433 - val_loss: 0.5372 - val_accuracy: 0.7468
Epoch 9/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6212 - accuracy: 0.6710 - val_loss: 0.5351 - val_accuracy: 0.7468
Epoch 10/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6066 - accuracy: 0.6824 - val_loss: 0.5225 - val_accuracy: 0.7403
Epoch 11/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5997 - accuracy: 0.6954 - val_loss: 0.5965 - val_accuracy: 0.7143
Epoch 12/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5943 - accuracy: 0.7003 - val_loss: 0.5158 - val_accuracy: 0.7273
Epoch 13/

The conclusion is you tune any hyperparameter using this keras tuner 